# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-01-13 08:17:33 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.7

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.6.3
Commit ae8452a9e0 (2021-09-23 17:34 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


In [3]:
import Pkg; Pkg.add("BenchmarkTools")

    Updating registry at `C:\Users\Konrad Grudzinski\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed HarfBuzz_jll ─── v2.8.1+1
   Installed Glib_jll ─────── v2.68.3+2
   Installed Ogg_jll ──────── v1.3.5+1
   Installed Cairo_jll ────── v1.16.1+1
   Installed Libffi_jll ───── v3.2.2+1
   Installed BenchmarkTools ─ v1.2.2
    Updating `C:\Users\Konrad Grudzinski\.julia\environments\v1.6\Project.toml`
  [6e4b80f9] + BenchmarkTools v1.2.2
    Updating `C:\Users\Konrad Grudzinski\.julia\environments\v1.6\Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.2.2
  [83423d85] ↑ Cairo_jll v1.16.1+0 ⇒ v1.16.1+1
  [7746bdde] ↑ Glib_jll v2.68.3+0 ⇒ v2.68.3+2
  [2e76f6c2] ↑ HarfBuzz_jll v2.8.1+0 ⇒ v2.8.1+1
  [e9f186c6] ↑ Libffi_jll v3.2.2+0 ⇒ v3.2.2+1
  [e7412a2a] ↑ Ogg_jll v1.3.5+0 ⇒ v1.3.5+1
  [9abbd945] + Profile
Precompiling project...
  ✓ Ogg_jll
  ✓ Libffi_jll
  ✓ BenchmarkTools
  ✓ libvorbis_jll
  ✓ Gl

In [4]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  149.660 ms (2 allocations: 32.00 MiB)


In [3]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    run(`nvidia-smi`)

    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
else
    println("No GPU found.")
end

No GPU found.


In [1]:
import Pkg

In [2]:
Pkg.add("DataFrames")
Pkg.add("Distances")
Pkg.add("StatsBase")
Pkg.add("Distributions")
Pkg.add("Zygote")
Pkg.add("Flux")
using DataFrames
using Distributions
using StatsBase
using Distances
using Zygote
using Flux

    Updating registry at `C:\Users\Konrad Grudzinski\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\Konrad Grudzinski\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\Konrad Grudzinski\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ SubClonalSelection
  1 dependency successfully precompiled in 22 seconds (156 already precompiled)
   Resolving package versions...
  No Changes to `C:\Users\Konrad Grudzinski\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\Konrad Grudzinski\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ SubClonalSelection
  1 dependency successfully precompiled in 16 seconds (156 already precompiled)
   Resolving package versions...
  No Changes to `C:\Users\Konrad Grudzinski\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\Konrad Grudzinski\.julia\environments\v1.6\Manifest.t

In [ ]:
counts(mutations,minimum(mutations):maximum(mutations))

In [81]:
a = [1,6,4,3,7,9,2]
b = [5,3,7,3]
b[5]

LoadError: BoundsError: attempt to access 4-element Vector{Int64} at index [5]

---

In [75]:
"""
    simulate(; <keyword arguments>)

Simulate a stochastic model of tumour growth with a single subclone introduced at a random time and with a random fitness advantage. Output return synthetic sequencing data.
...
## Arguments
- `read_depth = 200.0`: Mean read depth of the target data set
- `detectionlimit = 5/read_depth`: Ability to detect low frequency variants. Assumes 5 reads are needed to call a variant.
- `μ = 10.0`: Mutation rate per division per genome (this will timesed by ploidy for the mutation rate per cell). At each division a Poisson random variable with mean μ is sampled.
- `clonalmutations = 100.0`: Number of clonal mutations present in the first cell.
- `nclones = 1`: Number of subclones introduced
- `Nmax = 10^4`: Maximum population size.
- `ρ = 0.0`: Overdispersion parameter for beta-binomial model of sequencing data. ρ = 0.0 means model is binomial sampling
- `timefunction = exptime`: Function for KMC algorithm timestep. exptime returns an exponentially distributed random variable, if you would rather return the mean of the distribution define a function that returns 1. ie `returnone() = 1`.
- `ploidy = 2`: ploidy of the genome
- `d = 0.0`: Death rate of the thost population in the tumour
- `b = log(2)`: Birth rate of the population. Set to `log(2)` so that tumour doubles with each unit increase in t in the absence of cell death.
- `fixedmu = false`: If set to false number of mutations per division is fixed and not sampled from a poisson distribution.
...
"""
function simulate(; nclones = 1, ploidy = 2, read_depth = 100.0, detectionlimit = 5 ./ read_depth, clonalmutations = 100.0, μ = 10.0, d = 0.0, b = log(2), ρ = 0.0, Nmax = 10^4, s = repeat([1.0], inner = nclones), tevent = collect(1.0:0.5:100.0)[1:nclones], cellularity = 1.0, fixedmu = false, timefunction::Function = exptime, maxclonefreq = 200)

    nclones == length(s) || error("Number of clones is $(nclones), size of selection coefficient array is $(length(s)), these must be the same size ")
    nclones == length(tevent) || error("Number of clones is $(nclones), size of selection coefficient array is $(length(tevent)), these must be the same size ")
    IP = InputParameters(nclones,
    Nmax,
    detectionlimit,
    ploidy,
    read_depth,
    clonalmutations,
    s,
    (μ/2) * ploidy,
    b,
    d,
    tevent,
    ρ,
    cellularity,
    fixedmu,
    timefunction,
    maxclonefreq)

    #get simulation data
    simresult, IP = run1simulation(IP, 0.0, 1.0)

    #get sampled VAFs
    if IP.ρ > 0.0
        sampleddata = sampledhist(simresult.trueVAF, IP.Nmax, IP.ρ, detectionlimit = IP.detectionlimit, ploidy = IP.ploidy, read_depth = IP.read_depth, cellularity = IP.cellularity)
    else
        sampleddata = sampledhist(simresult.trueVAF, IP.Nmax, detectionlimit = IP.detectionlimit, ploidy = IP.ploidy, read_depth = IP.read_depth, cellularity = IP.cellularity)
    end

    return Simulation(IP, simresult, sampleddata)
end

mutable struct SampledData
    DF::DataFrame
    VAF::Array{Float64,1}
    counts::Array{Int64,1}
    depth::Array{Int64,1}
end

mutable struct Simulation
  input::InputParameters
  output::SimResult
  sampleddata::SampledData
end


###############################################################################

function betabinom(p, n, ρ)
    μ = p * n
    shape1 = ((μ / n) * ((1 / ρ) - 1))
    shape2 = abs(n * shape1/μ - shape1)
    return rand(Binomial(n, rand(Beta(shape1, shape2))))
end


function sampledhist(AF::Array{Float64, 1}, cellnum::Int64; detectionlimit = 0.1, ploidy = 2.0, read_depth = 100.0, cellularity = 1.0)

    AF = AF./ploidy
    AF = AF .* cellularity
    filter!(x -> x > detectionlimit * cellnum, AF)
    samp_percent = read_depth/cellnum
    #depth = rand(Binomial(cellnum,samp_percent), length(AF))
    depth = rand(Poisson(read_depth), length(AF))
    samp_alleles = map((n, p) -> rand(Binomial(n, p)), depth, AF/cellnum)

    VAF = samp_alleles./depth

    #data for histogram
    x = 0.005:0.01:1.005
    y = fit(Histogram, VAF, x, closed=:right)
    DFhist = DataFrame(VAF = x[1:end-1], freq = y.weights)

    SampledData(DFhist, VAF, samp_alleles, depth)
end

function sampledhist(AF, cellnum, ρ; detectionlimit = 0.1, ploidy = 2.0, read_depth = 100.0, cellularity = 1.0)

    AF = AF./ploidy
    AF = AF .* cellularity
    filter!(x -> x > detectionlimit * cellnum, AF)
    samp_percent = read_depth/cellnum
    #depth = rand(Binomial(cellnum, samp_percent), length(AF))
    depth = rand(Poisson(read_depth), length(AF))
    samp_alleles = map((x, y) -> betabinom(x, y, ρ), AF/cellnum, depth)
    VAF = samp_alleles./depth

    #data for histogram
    x = 0.005:0.01:1.005
    y = fit(Histogram, VAF, x, closed=:right)
    DFhist = DataFrame(VAF = x[1:end-1], freq = y.weights)

    SampledData(DFhist, VAF, samp_alleles, depth)
end

function getCDF(VAF::Array, step_size::Float64; fmin = 0.05, fmax = 0.7)
  #fast way to calculate CDF
  out = cumsum(fit(Histogram, VAF, fmin:step_size:fmax,closed=:left).weights[1:end - 1])
  out = out .- out[1]
  return out
end

getCDF (generic function with 1 method)

In [18]:
io = open("test.txt", "w") do io
  for x in cdf
    println(io, x)
  end
end

LoadError: MethodError: no method matching iterate(::typeof(cdf))
[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m) at range.jl:664
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, [91m::Int64[39m) at range.jl:664
[0m  iterate([91m::T[39m) where T<:Union{Base.KeySet{var"#s77", var"#s76"} where {var"#s77", var"#s76"<:Dict}, Base.ValueIterator{var"#s75"} where var"#s75"<:Dict} at dict.jl:693
[0m  ...

In [5]:
function get_parameters()
    nclones = rand(0:2)
    ploidy = rand(1:2)
    read_depth = rand(50:500)
    clonalmutations = rand(50:500)
    mu = rand(2:20)
    d = rand(Float64)/3
    b = rand(Float64) + 0.25
    cellularity = rand(Float64) + 0.1
    detectionlimit = rand(Float64)/40 + 0.1
    rho = rand(Float64)/4
    return Dict(:nclones=>nclones, :ploidy=>ploidy, :read_depth=>read_depth, :clonalmutations=>clonalmutations, :μ=>mu, :d=>d, :b=>b, :cellularity=>cellularity, :detectionlimit=>detectionlimit, :ρ=>rho)
end

function arrayed_params(params)
    return [params[:nclones], params[:ploidy], params[:read_depth], params[:clonalmutations], params[:μ], params[:d], params[:b], params[:cellularity], params[:detectionlimit], params[:ρ]]
end

arrayed_params (generic function with 1 method)

In [6]:
lr_g = 2e-4          # Learning rate of the generator network
lr_d = 2e-4          # Learning rate of the discriminator network
batch_size = 10     # batch size
num_epochs = 100       # Number of epochs to train for
output_period = 100  # Period length for plots of generator samples
n_features = 140     # Length of CDF vector
input_dim = 10     # Dimension of latent space
opt_dscr = ADAM(lr_d)# Optimizer for the discriminator
opt_gen = ADAM(lr_g) # Optimizer for the generator

ADAM(0.0002, (0.9, 0.999), IdDict{Any, Any}())

In [7]:
generator = Chain(Dense(input_dim, 256, x -> leakyrelu(x, 0.2f0)),
                    Dense(256, 512, x -> leakyrelu(x, 0.2f0)),
                    Dense(512, 1024, x -> leakyrelu(x, 0.2f0)),
                    Dense(1024, n_features, tanh))# |> gpu

Chain(Dense(10, 256, #23), Dense(256, 512, #24), Dense(512, 1024, #25), Dense(1024, 140, tanh))

In [8]:
discriminator = Chain(Dense(n_features, 1024, x -> leakyrelu(x, 0.2f0)),
                        Dropout(0.3),
                        Dense(1024, 512, x -> leakyrelu(x, 0.2f0)),
                        Dropout(0.3),
                        Dense(512, 256, x -> leakyrelu(x, 0.2f0)),
                        Dropout(0.3),
                        Dense(256, 1, sigmoid))# |> gpu

Chain(Dense(140, 1024, #29), Dropout(0.3), Dense(1024, 512, #30), Dropout(0.3), Dense(512, 256, #31), Dropout(0.3), Dense(256, 1, σ))

In [9]:
function train_dscr!(discriminator, real_data, fake_data)
    this_batch = size(real_data)[end] # Number of samples in the batch
    # Concatenate real and fake data into one big vector
    all_data = hcat(real_data, fake_data)

    # Target vector for predictions: 1 for real data, 0 for fake data.
    all_target = [1,0]# |> gpu;

    ps = Flux.params(discriminator)
    loss, pullback = Zygote.pullback(ps) do
        preds = discriminator(all_data)
        loss = Flux.Losses.binarycrossentropy(preds, all_target)
    end
    # To get the gradients we evaluate the pullback with 1.0 as a seed gradient.
    grads = pullback(1f0)

    # Update the parameters of the discriminator with the gradients we calculated above
    Flux.update!(opt_dscr, Flux.params(discriminator), grads)
    
    return loss 
end

train_dscr! (generic function with 1 method)

In [10]:
function train_gen!(discriminator, generator)
    # Sample noise
    parameters = get_parameters()# |> gpu;

    # Define parameters and get the pullback
    ps = Flux.params(generator)
    # Evaluate the loss function while calculating the pullback. We get the loss for free
    loss, back = Zygote.pullback(ps) do
        preds = discriminator(generator(arrayed_params(parameters)));
        loss = Flux.Losses.binarycrossentropy(preds, 1.) 
    end
    # Evaluate the pullback with a seed-gradient of 1.0 to get the gradients for
    # the parameters of the generator
    grads = back(1.0f0)
    Flux.update!(opt_gen, Flux.params(generator), grads)
    return loss
end

train_gen! (generic function with 1 method)

In [11]:
lossvec_gen = zeros(num_epochs)
lossvec_dscr = zeros(num_epochs)
n = 1

1

In [ ]:
println(n)
loss_sum_gen = 0.0f0
loss_sum_dscr = 0.0f0

for i in 1:batch_size
    parameters = get_parameters()
    real_data = getCDF(simulate(;parameters...).sampleddata.VAF, 0.005, fmax=0.755);

    # Train the discriminator
    fake_data = generator(arrayed_params(parameters))
    loss_dscr = train_dscr!(discriminator, real_data, fake_data)
    loss_sum_dscr += loss_dscr

    # Train the generator
    loss_gen = train_gen!(discriminator, generator)
    loss_sum_gen += loss_gen
end

# Add the per-sample loss of the generator and discriminator
lossvec_gen[n] = loss_sum_gen / batch_size
lossvec_dscr[n] = loss_sum_dscr / batch_size
n += 1

In [37]:
lossvec_gen, lossvec_dscr

([0.7297754983463378, 0.7016590070963671, 0.6406931500886153, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [7.648880723185786, 8.503946536243358, 8.513192774249358, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:
for n in 1:num_epochs
    println(n)
    loss_sum_gen = 0.0f0
    loss_sum_dscr = 0.0f0

    for i in 1:batch_size
        parameters = get_parameters()
        real_data = getCDF(simulate(;parameters...).sampleddata.VAF, 0.005, fmax=0.755);

        # Train the discriminator
        fake_data = generator(arrayed_params(parameters))
        loss_dscr = train_dscr!(discriminator, real_data, fake_data)
        loss_sum_dscr += loss_dscr

        # Train the generator
        loss_gen = train_gen!(discriminator, generator)
        loss_sum_gen += loss_gen
    end

    # Add the per-sample loss of the generator and discriminator
    lossvec_gen[n] = loss_sum_gen / batch_size
    lossvec_dscr[n] = loss_sum_dscr / batch_size
end 

1
2
3
4
5
6
7

In [76]:
function simulate2(; nclones = 1, ploidy = 2, read_depth = 100.0, detectionlimit = 5 ./ read_depth, clonalmutations = 100.0, μ = 10.0, d = 0.0, b = log(2), ρ = 0.0, Nmax = 10^4, s = repeat([1.0], inner = nclones), tevent = collect(1.0:0.5:100.0)[1:nclones], cellularity = 1.0, fixedmu = false, timefunction::Function = exptime, maxclonefreq = 200)

    nclones == length(s) || error("Number of clones is $(nclones), size of selection coefficient array is $(length(s)), these must be the same size ")
    nclones == length(tevent) || error("Number of clones is $(nclones), size of selection coefficient array is $(length(tevent)), these must be the same size ")
    IP = InputParameters(nclones,
    Nmax,
    detectionlimit,
    ploidy,
    read_depth,
    clonalmutations,
    s,
    (μ/2) * ploidy,
    b,
    d,
    tevent,
    ρ,
    cellularity,
    fixedmu,
    timefunction,
    maxclonefreq)

    #get simulation data
    simresult, IP = run1simulation(IP, 0.0, 1.0)

    return simresult, IP
end

simulate2 (generic function with 1 method)

In [100]:
#mutable struct definitions
abstract type Stem end

mutable struct CancerStemCell <: Stem
    mutations::Array{Int64,1}
    fitness::Int64
end

mutable struct CancerNonStemCell <: Stem
    mutations::Array{Int64,1}
    fitness::Int64
    Ndivs::Int64
end

mutable struct CancerCells
  stemcells::Array{CancerStemCell, 1}
  nonstemcells::Array{CancerNonStemCell, 1}
  ncells::Array{Int64, 1}
  α::Float64
  d::Float64
end

mutable struct RawResults
  cells::CancerCells
  Nvec::Array{Float64, 1}
  divisions::Array{Int64, 1}
end

mutable struct cancercell
    mutations::Array{Int64,1}
    fitness::Int64
end

mutable struct RawOutput
    Nvec::Array{Int64, 1}
    tvec::Array{Float64, 1}
    muts::Array{Int64, 1}
    cells::Array{cancercell, 1}
    birthrates::Array{Float64, 1}
    deathrates::Array{Float64, 1}
    clonetype::Array{Int64, 1}
    clonetime::Array{Float64, 1}
    subclonemutations::Array{Any, 1}
    cloneN::Array{Int64, 1}
    Ndivisions::Array{Int64, 1}
    aveDivisions::Array{Float64, 1}
end

mutable struct bdprocess
  N::Array{Int64, 1}
  t::Array{Float64, 1}
  clonefreq::Array{Float64, 1}
end

mutable struct SimResult
    clonefreq::Array{Float64,1}
    clonefreqp::Array{Float64,1}
    clonetime::Array{Float64,1}
    subclonemutations::Array{Int64,1}
    birthrates::Array{Float64,1}
    deathrates::Array{Float64,1}
    tend::Float64
    trueVAF::Array{Float64,1}
    cloneN::Array{Int64, 1}
    clonetype::Array{Int64, 1}
    Ndivisions::Array{Int64, 1}
    cells::Array{cancercell, 1}
    aveDivisions::Array{Float64, 1}
end

mutable struct InputParameters
    numclones::Int64
    Nmax::Int64
    detectionlimit::Float64
    ploidy::Int64
    read_depth::Float64
    clonalmutations::Int64
    selection::Array{Float64,1}
    μ::Float64
    b::Float64
    d::Float64
    tevent::Array{Float64,1}
    ρ::Float64
    cellularity::Float64
    fixedmu::Bool
    timefunction::Function
    maxclonefreq::Int64
end

mutable struct StemCellSimResult
    trueVAF::Array{Float64,1}
    cells::CancerCells
    N::Array{Float64, 1}
    divisions::Array{Float64, 1}
    stemcellfraction::Array{Float64,1}
end

###############################################################################

function newmutations(cancercell, μ, mutID)
    #function to add new mutations to cells based on μ
    numbermutations= 1
    cancercell.mutations = append!(cancercell.mutations, mutID:mutID+numbermutations-1)
    mutID = mutID + numbermutations
    return cancercell, mutID
end

function newmutationsinit(cancercell, μ, mutID)
    numbermutations = 0
    cancercell.mutations = append!(cancercell.mutations,mutID:mutID+numbermutations-1)
    mutID = mutID + numbermutations
    return cancercell, mutID
end

function initializesim(clonalmutations)

    #initialize time to zero
    t = 0.0
    tvec = Float64[]
    push!(tvec,t)

    #population starts with one cell
    N = 1
    Nvec = Int64[]
    push!(Nvec,N)

    #Initialize array of cell type that stores mutations for each cell and their fitness type
    #fitness type of 1 is the host population, lowest fitness
    cells = cancercell[]
    #sizehint!(cells, 10^5)
    push!(cells,cancercell([],1))

    #need to keep track of mutations, assuming infinite sites, new mutations will be unique,
    #we assign each new muation with a unique integer by simply counting up from one
    mutID = 1
    cells[1],mutID = newmutationsinit(cells[1],clonalmutations,mutID)

    return t,tvec,N,Nvec,cells,mutID
end

exptime() = - log(rand())
meantime() = 1

function copycell(cancercellold::cancercell)
  newcancercell::cancercell = cancercell(copy(cancercellold.mutations), copy(cancercellold.fitness))
end

function tumourgrow_birthdeath(b, d, Nmax, μ; numclones=1, clonalmutations = μ, s = [0.0], tevent=[0.0], maxclonefreq = 200, timefunction::Function = meantime)

    #set array of birthrates
    birthrates = [b]
    deathrates = [d]
    times = vcat(tevent, 0.0)
    #time is defined in terms of population doublings
    timesN = round.(Int64, vcat(exp.(log(2).*times[1:end-1]), 0.0))

    #depending on number of clones add birthrates to model, fitness is randomly distributed between death and birth rates
    for i in 1:numclones
        push!(deathrates, rand() * deathrates[1])
        push!(birthrates,(1 + s[i]) * (birthrates[1] - deathrates[1]) + deathrates[i + 1])
    end

    #Rmax starts with b + d and changes once a fitter mutant is introduced, this ensures that
    # b and d have correct units
    Rmax = b + d

    #initialize arrays and parameters
    t,tvec,N,Nvec,cells,mutID = initializesim(clonalmutations)
    muts = Int64[]
    push!(muts,mutID)

    #we only want to introduce mutant once so have variable that keeps track of how many mutants have been introduced, keep track of which type of which clone aquires new clone
    fitmutant = 1
    clonetype = Int64[]
    clonetime = Float64[]
    subclonemutations = Any[]
    cloneN = Int64[]
    Ndivisions = Int64[]
    aveDivisions = Float64[]

    clonefreq = zeros(Int64, numclones + 1)
    clonefreq[1] = 1

    executed = false
    changemutrate = broadcast(!, BitArray(undef, numclones + 1))

    while N < Nmax

        #pick a random cell
        randcell = rand(1:N)
        r = rand(Uniform(0,Rmax))
        Nt = N

        #birth event if r<birthrate, access correct birthrate from cells array
        if r < birthrates[cells[randcell].fitness]

            #population increases by one
            N = N + 1
            #copy cell and mutations for cell that reproduces
            #push!(cells, deepcopy(cells[randcell]))
            push!(cells, copycell(cells[randcell]))
            #add new mutations to both new cells
            if μ > 0.0
              cells[randcell],mutID = newmutations(cells[randcell],μ,mutID)
              cells[end],mutID = newmutations(cells[end],μ,mutID)
            end
            push!(muts,mutID)
            clonefreq[cells[randcell].fitness] = clonefreq[cells[randcell].fitness] + 1
            push!(Nvec, N)
            Δt =  1/(Rmax * Nt) .* timefunction()
            t = t + Δt
            push!(tvec,t)

            #if population time is tevent, cell is mutated into fitter cell
            if N >= timesN[fitmutant]
                if fitmutant != numclones + 1
                    #one mutant turns into another "type" so decreases in frequency

                    clonefreq[cells[randcell].fitness] = clonefreq[cells[randcell].fitness] - 1
                    #keep track of how many clones
                    fitmutant += 1
                    println(fitmutant)
                    push!(clonetype, cells[randcell].fitness)
                    #change one mutant to fitter mutant
                    cells[randcell].fitness = fitmutant
                    #new type increases in frequency
                    clonefreq[cells[randcell].fitness] = clonefreq[cells[randcell].fitness] + 1

                    #change Rmax given that we now have a new fitter mutant
                    Rmax = maximum(birthrates[1:fitmutant]) + maximum(deathrates[1:fitmutant])

                    push!(clonetime, t)
                    push!(subclonemutations, deepcopy(cells[randcell].mutations))
                    push!(cloneN, N)
                    push!(Ndivisions, length(cells[randcell].mutations))
                    divs = map(x -> length(x.mutations), cells)
                    push!(aveDivisions, mean(divs))

                end
            end
        end

        if (birthrates[cells[randcell].fitness] + deathrates[cells[randcell].fitness]) <= r
          push!(Nvec, N)
          Δt =  1/(Rmax * Nt) * timefunction()
          t = t + Δt
          push!(tvec,t)
        end

        #death event if b<r<b+d
        if (birthrates[cells[randcell].fitness] <= r < birthrates[cells[randcell].fitness] + deathrates[cells[randcell].fitness])

            #population decreases by 1
            N = N - 1
            #frequency of cell type decreases
            clonefreq[cells[randcell].fitness] = clonefreq[cells[randcell].fitness] - 1
            #remove deleted cell
            deleteat!(cells,randcell)
            push!(Nvec,N)
            Δt =  1/(Rmax * Nt) * timefunction()
            t = t + Δt
            push!(tvec,t)
        end

        #every cell dies reinitialize simulation
        if (N == 0)
            t,tvec,N,Nvec,cells,mutID = initializesim(clonalmutations)
            muts = Int64[]
            push!(muts,mutID)
        end

        if (executed == false) && ((clonefreq.>maxclonefreq) == changemutrate)
            #if population of all clones is sufficiently large no new mutations
            #are acquired, can use this approximation as only mutations above 1%
            # frequency can be reliably detected
            μ = 0
            executed = true
        end

    end
    return RawOutput(Nvec, tvec, muts, cells, birthrates, deathrates, clonetype, clonetime, subclonemutations, cloneN, Ndivisions, aveDivisions)
end

function cellsconvert(cells)
    #convert from array of cell types to one array with mutations and one array with cell fitness

    fitness = zeros(Int64,length(cells))
    mutations = Int64[]
    sizehint!(mutations, length(cells) * 10)

    for i in 1:length(cells)
        append!(mutations,cells[i].mutations)
        fitness[i] = cells[i].fitness
    end

    return mutations, fitness
end

function allelefreq(mutations, cellnum)
    #create dictionary that maps mutation ID to allele frequency
    f = counts(mutations,minimum(mutations):maximum(mutations))
    muts = collect(minimum(mutations):maximum(mutations))
    idx = f .> 0.01
    f = map(Float64, f[idx])
    muts = muts[idx]
    Dict{Int64, Float64}(muts[i]::Int64 => f[i]::Float64 for i in 1:length(f))
end

function getresults(tevent::Array{Float64, 1}, s::Array{Float64, 1}, b, d, μ, Nmax; ploidy = 2, clonalmutations = 100, nc = 0, timefunction = exptime, maxclonefreq = 200)

    #Nvec,tvec,mvec,cells,br,dr,ct,clonetime
    sresult = tumourgrow_birthdeath(b, d, Nmax, μ; numclones = nc, s = s, tevent = tevent, clonalmutations = 0, timefunction = timefunction, maxclonefreq = maxclonefreq);
    M,fitness = cellsconvert(sresult.cells)

    return M, fitness, sresult.tvec[end], sresult.clonetime, sresult.subclonemutations, sresult.birthrates, sresult.deathrates, sresult.cloneN, sresult.clonetype, sresult.Ndivisions, sresult.cells, sresult.aveDivisions

end

function allelefreqexpand(AFDict, μ, subclonemutations; fixedmu = false)

  #expand allele frequency given mutation rate and calculate number of mutations in the subclones
  #subclonemutations = convert(Array{Array{Int64,1},1}, subclonemutations)
  if fixedmu == false
    cmuts = zeros(Int64, length(subclonemutations))
    mutfreqs = collect(values(AFDict))
    mutids = collect(keys(AFDict))
    mutations = rand(Poisson(μ), length(mutfreqs))
    AFnew = zeros(Int64, sum(mutations))

    for i in 1:length(cmuts)
      idx = findall((in)(mutids), subclonemutations[i])
      cmuts[i] = sum(mutations[idx])
    end

    j = 0
    for f in 1:length(mutfreqs)
        AFnew[(j + 1): j + mutations[f]] = fill(mutfreqs[f], mutations[f])
        j = j + mutations[f]
    end

  else
    mutfreqs = collect(values(AFDict))
    mutids = collect(keys(AFDict))
    μint = round(Int64, μ)
    mutations = fill(Int64(μ), length(mutfreqs))
    AFnew = zeros(Int64, sum(mutations))
    cmuts = zeros(Int64, length(subclonemutations))

    for i in 1:length(cmuts)
      idx = findin(mutids, subclonemutations[i])
      cmuts[i] = sum(mutations[idx])
    end

    j = 0
    for f in 1:length(mutfreqs)
      AFnew[(j + 1): j + mutations[f]] = fill(mutfreqs[f], mutations[f])
      j = j + mutations[f]
    end
  end

  return AFnew, cmuts
end

function calculateclonefreq(pctfit, cmuts, clonetype)

  cf = deepcopy(pctfit)
  parent = clonetype
  cloneid = collect(1:length(clonetype))
  prev = 0
  for i in reverse(cloneid)[1:end-1]
    if parent[i] == 0
        continue
    end
    cf[parent[i]] = cf[parent[i]] + cf[i]
    cmuts[i] = cmuts[i] - cmuts[parent[i]]
  end

  return cf, cmuts
end


function run1simulation(IP::InputParameters, minclonesize, maxclonesize)

    M, fitness, tend, clonetime, subclonemutations, br, dr, cloneN, clonetype, Ndivisions, cells, aveDivisions = getresults(IP.tevent, IP.selection, IP.b, IP.d, IP.μ, IP.Nmax; ploidy = IP.ploidy, clonalmutations = IP.clonalmutations, nc = IP.numclones, timefunction = IP.timefunction, maxclonefreq = IP.maxclonefreq)

    if length(clonetime)!= IP.numclones

        IP.numclones = length(clonetime)
        IP.tevent = IP.tevent[1:IP.numclones]
        IP.selection = IP.selection[1:IP.numclones]
        br = br[1:IP.numclones]
        dr = dr[1:IP.numclones]
        clonetype = clonetype[1:IP.numclones]

    end

    AF = allelefreq(M, IP.Nmax)
    #println(AF)
    AF, cmuts = allelefreqexpand(AF, IP.μ, subclonemutations, fixedmu = IP.fixedmu)
    #println(AF)
    #println(cmuts)
    prepend!(AF, fill(Float64(IP.Nmax), IP.clonalmutations))
    
    pctfit=Float64[]
    for i in 1:IP.numclones push!(pctfit,sum(fitness.==(i+1))/IP.Nmax) end

    #remove clones that have frequency < detectionlimit
    if length(pctfit) > 1
      clonefreq, cmuts = calculateclonefreq(pctfit, cmuts, clonetype .- 1)
      !(sum(clonefreq.>1.0) > 0) || error("There is a clone with frequency greater than 1, this should be impossible ($(clonefreq)), $(clonetype), $(pctfit)")
    else
      clonefreq = pctfit
    end

    if VERSION < v"0.6-"
        detectableclones = (clonefreq.>minclonesize) & (clonefreq.<maxclonesize)  # Deprecated as of v0.6
    else
        detectableclones = (clonefreq.>minclonesize) .& (clonefreq.<maxclonesize)
    end
    #println(clonefreq)
    #println(detectableclones)
    clonefreq = clonefreq[detectableclones]
    
    if sum(detectableclones) != IP.numclones
        IP.numclones = sum(detectableclones)
        IP.tevent = IP.tevent[detectableclones]
        IP.selection = IP.selection[detectableclones]
        clonetype = clonetype[detectableclones]
        pushfirst!(detectableclones, true)
        detectableclones = detectableclones[1:length(br)]
        br = br[detectableclones]
        dr = dr[detectableclones]
    end

    return SimResult(clonefreq, pctfit, clonetime, cmuts, br, dr, tend, AF, cloneN, clonetype .- 1, Ndivisions, cells, aveDivisions), IP
end

###############################################################################


run1simulation (generic function with 1 method)

In [123]:
results = simulate2()[1]

2


SimResult([0.9935], [0.9935], [1.0858122367093466], [6], [0.6931471805599453, 1.3862943611198906], [0.0, 0.0], 8.243210829244488, [10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0  …  2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], [2], [0], [1], cancercell[cancercell([1, 3, 5, 7, 11, 33, 49, 91, 153, 233, 413, 423, 1589, 4709, 7381], 2), cancercell([2, 53, 119, 305, 18775], 1), cancercell([1, 4, 23, 45, 851, 12645, 14643, 17781], 2), cancercell([1, 3, 6, 13, 131, 1421, 2523, 6195, 6611, 8551, 12285, 16589], 2), cancercell([1, 3, 5, 8, 9, 29, 87, 453, 541, 889, 1033, 6551], 2), cancercell([1, 3, 5, 8, 10, 19, 79, 129, 219, 281, 299, 877, 2003, 3353, 19953], 2), cancercell([1, 3, 5, 7, 12, 21, 149, 287, 583, 1331, 1411, 3575, 4759], 2), cancercell([1, 3, 6, 14, 15, 17, 313, 931, 1093], 2), cancercell([1, 3, 6, 14, 16, 457, 2259, 7729, 9297, 12565], 2), cancercell([1, 3, 6, 14, 15, 18, 145, 181, 505, 573, 5279, 7565, 12119], 2)  …  cancercell([1

In [168]:
@time simulate();

2
  0.021376 seconds (148.83 k allocations: 23.710 MiB)


In [126]:
results.sampleddata

SampledData(100×2 DataFrame
│ Row │ VAF     │ freq  │
│     │ Float64 │ Int64 │
├─────┼─────────┼───────┤
│ 1   │ 0.005   │ 1     │
│ 2   │ 0.015   │ 3     │
│ 3   │ 0.025   │ 1     │
│ 4   │ 0.035   │ 4     │
│ 5   │ 0.045   │ 11    │
│ 6   │ 0.055   │ 5     │
│ 7   │ 0.065   │ 15    │
│ 8   │ 0.075   │ 11    │
│ 9   │ 0.085   │ 12    │
│ 10  │ 0.095   │ 14    │
⋮
│ 90  │ 0.895   │ 0     │
│ 91  │ 0.905   │ 0     │
│ 92  │ 0.915   │ 0     │
│ 93  │ 0.925   │ 0     │
│ 94  │ 0.935   │ 0     │
│ 95  │ 0.945   │ 0     │
│ 96  │ 0.955   │ 0     │
│ 97  │ 0.965   │ 0     │
│ 98  │ 0.975   │ 0     │
│ 99  │ 0.985   │ 0     │
│ 100 │ 0.995   │ 0     │, [0.5714285714285714, 0.5573770491803278, 0.43564356435643564, 0.4594594594594595, 0.3684210526315789, 0.41964285714285715, 0.4563106796116505, 0.5196078431372549, 0.4945054945054945, 0.4479166666666667  …  0.20909090909090908, 0.13131313131313133, 0.15, 0.17204301075268819, 0.1926605504587156, 0.17307692307692307, 0.2, 0.19626168224299065, 0.1